## Bikes CNN for Price Regression

### Imports

--------------

We use Keras to build and train our model, with a Tensorflow backend. We use scikit-learn for evaluation metrics.

In [1]:
import csv
import math

import matplotlib.pyplot as plt
%matplotlib inline

import numpy as np

from keras import applications
from keras.callbacks import ModelCheckpoint
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model, load_model
from keras.layers import Dropout, Flatten, Dense, Input
from keras.initializers import glorot_uniform
from keras.applications.vgg16 import preprocess_input

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

Using TensorFlow backend.


In [2]:
import keras.backend as K

from keras.models import Model
from keras.layers import Input, Flatten, Dropout, Concatenate, Activation
from keras.layers import Convolution2D, MaxPooling2D, AveragePooling2D
from keras.layers import GlobalMaxPooling2D, GlobalAveragePooling2D

from keras.applications.imagenet_utils import decode_predictions
from keras.applications.imagenet_utils import preprocess_input
from keras_applications.imagenet_utils import _obtain_input_shape
from keras.utils.data_utils import get_file

WEIGHTS_PATH = 'https://github.com/wohlert/keras-squeezenet/releases/download/v0.1/squeezenet_weights.h5'

def _fire(x, filters, name="fire"):
    sq_filters, ex1_filters, ex2_filters = filters
    squeeze = Convolution2D(sq_filters, (1, 1), activation='relu', padding='same', name=name + "squeeze1x1")(x)
    expand1 = Convolution2D(ex1_filters, (1, 1), activation='relu', padding='same', name=name + "expand1x1")(squeeze)
    expand2 = Convolution2D(ex2_filters, (3, 3), activation='relu', padding='same', name=name + "expand3x3")(squeeze)
    x = Concatenate(axis=-1, name=name)([expand1, expand2])
    return x

def SqueezeNet(include_top=True, weights="imagenet", input_tensor=None, input_shape=None, pooling=None, classes=1000):

    if weights not in {'imagenet', None}:
        raise ValueError('The `weights` argument should be either '
                         '`None` (random initialization) or `imagenet` '
                         '(pre-training on ImageNet).')

    if weights == 'imagenet' and include_top and classes != 1000:
        raise ValueError('If using `weights` as imagenet with `include_top`'
                         ' as true, `classes` should be 1000')
    # Determine proper input shape
    input_shape = _obtain_input_shape(input_shape,
                                      default_size=224,
                                      min_size=48,
                                      data_format=K.image_data_format(),
                                      require_flatten=include_top)

    if input_tensor is None:
        img_input = Input(shape=input_shape)
    else:
        if not K.is_keras_tensor(input_tensor):
            img_input = Input(tensor=input_tensor, shape=input_shape)
        else:
            img_input = input_tensor

    x = Convolution2D(64, kernel_size=(3, 3), strides=(2, 2), padding="same", activation="relu", name='conv1')(img_input)
    x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), name='maxpool1', padding="valid")(x)

    x = _fire(x, (16, 64, 64), name="fire2")
    x = _fire(x, (16, 64, 64), name="fire3")

    x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), name='maxpool3', padding="valid")(x)

    x = _fire(x, (32, 128, 128), name="fire4")
    x = _fire(x, (32, 128, 128), name="fire5")

    x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), name='maxpool5', padding="valid")(x)

    x = _fire(x, (48, 192, 192), name="fire6")
    x = _fire(x, (48, 192, 192), name="fire7")

    x = _fire(x, (64, 256, 256), name="fire8")
    x = _fire(x, (64, 256, 256), name="fire9")

    if include_top:
        x = Dropout(0.5, name='dropout9')(x)

        x = Convolution2D(classes, (1, 1), padding='valid', name='conv10')(x)
        x = AveragePooling2D(pool_size=(13, 13), name='avgpool10')(x)
        x = Flatten(name='flatten10')(x)
        x = Activation("softmax", name='softmax')(x)
    else:
        if pooling == "avg":
            x = GlobalAveragePooling2D(name="avgpool10")(x)
        else:
            x = GlobalMaxPooling2D(name="maxpool10")(x)

    model = Model(img_input, x, name="squeezenet")

    if weights == 'imagenet':
        weights_path = get_file('squeezenet_weights.h5',
                                WEIGHTS_PATH,
                                cache_subdir='models')

        model.load_weights(weights_path)

    return model

### Load data on file paths and prices, as well as train/test split

--------------------

We load the locations of the images, as well as the train/test split. We use a 90/10 split, which we share across the neural network and baselines.

In [3]:
prices = []
image_paths = []

data_path = "../datasets/cars_im/"
with open("../datasets/cars_filtered.csv") as file:
    reader = csv.reader(file)
    i = -1
    for row in reader:
        i += 1
        index = row[0]
        name = row[1]
        msrp = int(row[3])
        
        image_path = data_path + index + '.jpg'
        image_paths.append(image_path)
        prices.append(int(msrp))

train_indices = np.load("cars_train_indices.npy")
test_indices = np.load("cars_test_indices.npy")
print(train_indices.shape)
print(test_indices.shape)

(1330,)
(147,)


### Initialize Image Generator

Due to the size of our dataset (>20,000 images), we cannot read all images into memory. Thus, we write our own image generator, which is a Python generator that reads images a minibatch at a time, preprocessing them and returning the input data and price labels as input to the neural network.

In [4]:
def image_generator(indices, batch_size):

    num_batches = int(len(indices) / batch_size)
    
    while True:
        for batch_i in range(num_batches):
            if batch_i == num_batches - 1:
                # special case: return as many as possible
                start_i = batch_i * batch_size
                batch_indices = indices[start_i:]
                
                X = np.zeros((len(batch_indices), 224, 224, 3))
                Y = np.zeros((len(batch_indices), 1))
            
            else:
                start_i = batch_i * batch_size
                end_i = start_i + batch_size

                batch_indices = indices[start_i:end_i]

                X = np.zeros((batch_size, 224, 224, 3))
                Y = np.zeros((batch_size, 1))
            
            for i, index in enumerate(batch_indices):
                img = image.load_img(image_paths[index], target_size=(224, 224))
                X[i, :, :, :] = image.img_to_array(img)                
                Y[i] = prices[index]
            
            # use vgg16 preprocessing
            X = preprocess_input(X)
            
            yield (X, Y)

### Hyperparameters

We tune hyperparameters using grid search and random search, modifying one hyperparameter at a time while keeping the others constant.

In [7]:
# Hyperparameters

num_settings = 1

hp_dropout = [0.5] * num_settings

#RMSprop
hp_lr = [0.001] * num_settings
hp_rho = [0.9] * num_settings
hp_epsilon = [1e-07] * num_settings
hp_decay = [0.0] * num_settings

# Number of hidden units
hp_hidden = [256] * num_settings

# Minibatch size
hp_mbsize = [64] * num_settings

num_epochs = 1000

In [11]:
# store the results of each setting
train_losses = np.zeros(num_settings)
dev_losses = np.zeros(num_settings)

for setting in range(num_settings):
    model = SqueezeNet(include_top=True)
    
    model.layers.pop()
    model.layers.pop()
    model.layers.pop()
    model.layers.pop()
    model.layers.pop()
    
    # build a classifier model to put on top of the convolutional model
    top_model = Sequential()
    top_model.add(Convolution2D(256, (1, 1), padding='valid', name='top_conv', input_shape=(model.layers[-1].output_shape[1:])))
    top_model.add(AveragePooling2D(pool_size=(5, 5), name='top_avgpool'))
    top_model.add(Flatten(input_shape=(model.layers[-1].output_shape[1:]),name='top_flatten'))
    top_model.add(Dropout(hp_dropout[setting], name='top_dropout'))
    top_model.add(Dense(hp_hidden[setting], activation='relu', kernel_initializer='glorot_uniform', name='top_dense'))
    top_model.add(Dense(1, activation='linear', name='output', kernel_initializer='glorot_uniform'))
    
    # add the model on top of the convolutional base
    new_model = Model(inputs= model.input, outputs = top_model(model.layers[-1].output))
    
    # set the first 19 layers (up to the last conv block)
    # to non-trainable (weights will not be updated)
#     for layer in new_model.layers[:-1]:
#         layer.trainable = False
    
#     new_model.summary()
    
    # RMSprop optimizer
    new_model.compile(loss='mean_squared_error',
                      optimizer=optimizers.RMSprop(
                              lr=hp_lr[setting], 
                              rho=hp_rho[setting], 
                              epsilon=hp_epsilon[setting], 
                              decay=hp_decay[setting]),
                     metrics=['mean_absolute_error'])
    
    checkpoint_path = 'output/cars-cnn-SqueezeNet_Vanilla-best.hdf5'
    
    # keep a checkpoint
    checkpoint = ModelCheckpoint(checkpoint_path,
                                monitor='val_loss',
                                save_best_only=True,
                                mode='min')
    
    
    minibatch_size = hp_mbsize[setting]

    train_steps = math.ceil(len(train_indices) / minibatch_size)
    test_steps = math.ceil(len(test_indices) / minibatch_size)

    # fine-tune the model
    history = new_model.fit_generator(
        image_generator(train_indices, minibatch_size),
        steps_per_epoch=train_steps,
        epochs=num_epochs,
        validation_data=image_generator(test_indices, minibatch_size),
        nb_val_samples=test_steps,
        callbacks=[checkpoint])
    
   
    print("==========")

C:\Users\Richard\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:63: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
C:\Users\Richard\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:63: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<generator..., steps_per_epoch=21, validation_steps=3, epochs=1000, callbacks=[<keras.ca..., validation_data=<generator...)`


Epoch 1/1000
21/21 [==============================] - 6s 278ms/step - loss: 16506311495.8550 - mean_absolute_error: 68643.4113 - val_loss: 6284142744.8720 - val_mean_absolute_error: 39908.4651
Epoch 2/1000
21/21 [==============================] - 4s 196ms/step - loss: 7748241218.2956 - mean_absolute_error: 44716.1580 - val_loss: 6105848244.3130 - val_mean_absolute_error: 35392.1146
Epoch 3/1000
21/21 [==============================] - 4s 189ms/step - loss: 10603336228.2113 - mean_absolute_error: 49397.8872 - val_loss: 6317723495.1280 - val_mean_absolute_error: 39005.0199
Epoch 4/1000
21/21 [==============================] - 4s 182ms/step - loss: 8654531307.9160 - mean_absolute_error: 49057.6811 - val_loss: 10607264215.9304 - val_mean_absolute_error: 94018.0300
Epoch 5/1000
21/21 [==============================] - 4s 179ms/step - loss: 9410522041.1217 - mean_absolute_error: 53932.7269 - val_loss: 7557868927.3934 - val_mean_absolute_error: 48808.5176
Epoch 6/1000
21/21 [=================

21/21 [==============================] - 4s 176ms/step - loss: 1104296509.9799 - mean_absolute_error: 19920.0882 - val_loss: 567221774.5592 - val_mean_absolute_error: 16795.6488
Epoch 44/1000
21/21 [==============================] - 4s 184ms/step - loss: 1593602471.8091 - mean_absolute_error: 22700.9702 - val_loss: 638856883.7565 - val_mean_absolute_error: 14996.9046
Epoch 45/1000
21/21 [==============================] - 4s 177ms/step - loss: 2466327335.2894 - mean_absolute_error: 24588.9583 - val_loss: 1676053742.4076 - val_mean_absolute_error: 23326.0489
Epoch 46/1000
21/21 [==============================] - 4s 175ms/step - loss: 1835743156.2656 - mean_absolute_error: 22617.3385 - val_loss: 1905957916.9391 - val_mean_absolute_error: 21144.1054
Epoch 47/1000
21/21 [==============================] - 4s 180ms/step - loss: 1613324288.5801 - mean_absolute_error: 21874.6333 - val_loss: 2025311910.2180 - val_mean_absolute_error: 23204.4893
Epoch 48/1000
21/21 [==============================

Epoch 86/1000
21/21 [==============================] - 4s 180ms/step - loss: 1102405589.9643 - mean_absolute_error: 17602.1680 - val_loss: 1409900886.8174 - val_mean_absolute_error: 17827.9445
Epoch 87/1000
21/21 [==============================] - 4s 182ms/step - loss: 1865691613.6398 - mean_absolute_error: 19151.4290 - val_loss: 814232902.9763 - val_mean_absolute_error: 18868.5486
Epoch 88/1000
21/21 [==============================] - 4s 182ms/step - loss: 1018658272.3527 - mean_absolute_error: 16355.0874 - val_loss: 470597896.9043 - val_mean_absolute_error: 11753.3361
Epoch 89/1000
21/21 [==============================] - 4s 175ms/step - loss: 1326257224.3203 - mean_absolute_error: 18887.3664 - val_loss: 801002584.8720 - val_mean_absolute_error: 17355.3071
Epoch 90/1000
21/21 [==============================] - 4s 182ms/step - loss: 952428200.2254 - mean_absolute_error: 16403.7125 - val_loss: 563306348.5217 - val_mean_absolute_error: 15867.1939
Epoch 91/1000
21/21 [===================

21/21 [==============================] - 4s 179ms/step - loss: 1368566693.8453 - mean_absolute_error: 17440.0636 - val_loss: 639292030.8870 - val_mean_absolute_error: 12966.3335
Epoch 129/1000
21/21 [==============================] - 4s 189ms/step - loss: 705774213.7013 - mean_absolute_error: 13852.1409 - val_loss: 326009074.1991 - val_mean_absolute_error: 11443.5840
Epoch 130/1000
21/21 [==============================] - 5s 215ms/step - loss: 1031591399.3301 - mean_absolute_error: 15601.3420 - val_loss: 410838039.9304 - val_mean_absolute_error: 14263.7212
Epoch 131/1000
21/21 [==============================] - 4s 198ms/step - loss: 1155723678.3989 - mean_absolute_error: 15309.3044 - val_loss: 536362688.6066 - val_mean_absolute_error: 13580.5705
Epoch 132/1000
21/21 [==============================] - 4s 189ms/step - loss: 734681860.0381 - mean_absolute_error: 13961.5496 - val_loss: 245672579.4783 - val_mean_absolute_error: 10255.7067
Epoch 133/1000
21/21 [==============================

...

Epoch 816/1000
21/21 [==============================] - 4s 178ms/step - loss: 537265628.0496 - mean_absolute_error: 10458.1746 - val_loss: 418790352.1391 - val_mean_absolute_error: 9865.0675
Epoch 817/1000
21/21 [==============================] - 4s 176ms/step - loss: 518787505.0132 - mean_absolute_error: 10338.0079 - val_loss: 259287556.1706 - val_mean_absolute_error: 9647.8106
Epoch 818/1000
21/21 [==============================] - 4s 177ms/step - loss: 426653309.7462 - mean_absolute_error: 9377.8017 - val_loss: 315769082.9913 - val_mean_absolute_error: 10451.6784
Epoch 819/1000
21/21 [==============================] - 4s 179ms/step - loss: 450123236.0981 - mean_absolute_error: 10140.2738 - val_loss: 237295519.3934 - val_mean_absolute_error: 9130.4316
Epoch 820/1000
21/21 [==============================] - 4s 180ms/step - loss: 565131888.3264 - mean_absolute_error: 10326.2543 - val_loss: 162669638.5391 - val_mean_absolute_error: 8719.2712
Epoch 821/1000
21/21 [=======================

Epoch 859/1000
21/21 [==============================] - 4s 177ms/step - loss: 496225832.3232 - mean_absolute_error: 10447.1605 - val_loss: 155270437.0047 - val_mean_absolute_error: 7777.5051
Epoch 860/1000
21/21 [==============================] - 4s 181ms/step - loss: 695048202.5162 - mean_absolute_error: 10480.7713 - val_loss: 157192068.4522 - val_mean_absolute_error: 7303.2660
Epoch 861/1000
21/21 [==============================] - 4s 177ms/step - loss: 372782558.6081 - mean_absolute_error: 9497.4075 - val_loss: 127434391.6588 - val_mean_absolute_error: 7191.1198
Epoch 862/1000
21/21 [==============================] - 4s 179ms/step - loss: 518880245.4579 - mean_absolute_error: 10653.9705 - val_loss: 477069749.1478 - val_mean_absolute_error: 15021.4653
Epoch 863/1000
21/21 [==============================] - 4s 177ms/step - loss: 446707739.5143 - mean_absolute_error: 9950.4021 - val_loss: 920832907.2227 - val_mean_absolute_error: 18739.5184
Epoch 864/1000
21/21 [=======================

Epoch 902/1000
21/21 [==============================] - 4s 175ms/step - loss: 438427836.3751 - mean_absolute_error: 9755.2102 - val_loss: 511158482.9217 - val_mean_absolute_error: 13408.2885
Epoch 903/1000
21/21 [==============================] - 4s 178ms/step - loss: 520388386.9362 - mean_absolute_error: 10375.5326 - val_loss: 627005283.1848 - val_mean_absolute_error: 14629.9993
Epoch 904/1000
21/21 [==============================] - 4s 178ms/step - loss: 453155429.3085 - mean_absolute_error: 9510.4523 - val_loss: 569749818.9913 - val_mean_absolute_error: 13076.1450
Epoch 905/1000
21/21 [==============================] - 4s 176ms/step - loss: 524332371.9572 - mean_absolute_error: 10486.0495 - val_loss: 383155213.3460 - val_mean_absolute_error: 12841.8424
Epoch 906/1000
21/21 [==============================] - 4s 178ms/step - loss: 500135969.3006 - mean_absolute_error: 10452.2595 - val_loss: 327619784.6261 - val_mean_absolute_error: 10699.5411
Epoch 907/1000
21/21 [====================

Epoch 945/1000
21/21 [==============================] - 4s 177ms/step - loss: 509648704.7168 - mean_absolute_error: 10360.7042 - val_loss: 260899491.0332 - val_mean_absolute_error: 11435.7780
Epoch 946/1000
21/21 [==============================] - 4s 176ms/step - loss: 604179973.7874 - mean_absolute_error: 10720.6251 - val_loss: 1070133220.1739 - val_mean_absolute_error: 12445.9139
Epoch 947/1000
21/21 [==============================] - 4s 176ms/step - loss: 565001416.7278 - mean_absolute_error: 10677.5654 - val_loss: 190682278.3697 - val_mean_absolute_error: 7400.0708
Epoch 948/1000
21/21 [==============================] - 4s 176ms/step - loss: 555409553.9173 - mean_absolute_error: 9997.9934 - val_loss: 303768134.9565 - val_mean_absolute_error: 8975.0022
Epoch 949/1000
21/21 [==============================] - 4s 182ms/step - loss: 402982735.3280 - mean_absolute_error: 9146.9613 - val_loss: 188409079.2796 - val_mean_absolute_error: 8496.9687
Epoch 950/1000
21/21 [======================

Epoch 988/1000
21/21 [==============================] - 4s 177ms/step - loss: 467063321.1206 - mean_absolute_error: 9914.9013 - val_loss: 393317365.1478 - val_mean_absolute_error: 12583.2132
Epoch 989/1000
21/21 [==============================] - 4s 176ms/step - loss: 465882037.9450 - mean_absolute_error: 10886.7222 - val_loss: 215567315.0332 - val_mean_absolute_error: 9174.3172
Epoch 990/1000
21/21 [==============================] - 4s 182ms/step - loss: 660771256.1795 - mean_absolute_error: 10520.8333 - val_loss: 321958079.7217 - val_mean_absolute_error: 9174.5973
Epoch 991/1000
21/21 [==============================] - 4s 178ms/step - loss: 411924240.2723 - mean_absolute_error: 9003.8925 - val_loss: 221826352.1517 - val_mean_absolute_error: 9022.1701
Epoch 992/1000
21/21 [==============================] - 4s 175ms/step - loss: 458600989.7728 - mean_absolute_error: 9717.7831 - val_loss: 150836154.4348 - val_mean_absolute_error: 7375.2175
Epoch 993/1000
21/21 [=========================

In [ ]:
# store the training and dev losses for the last epoch (current model)
train_losses[setting] = history.history['loss'][-1]
dev_losses[setting] = history.history['val_loss'][-1]


plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['train', 'test'], loc='upper right')
plt.show()

### Evaluation Metrics

In [ ]:
# get predictions on each batch yielded the validation generator.

validation_generator = image_generator(test_indices, minibatch_size)

predicted = []
actual = []

for step in range(test_steps):
    X, Y = next(validation_generator)
    curr_pred = new_model.predict(X)
    for entry in curr_pred:
        predicted.append(entry)
    for entry in Y:
        actual.append(entry)

In [ ]:
predicted = np.array(predicted)
actual = np.array(actual)

MSE = mean_squared_error(predicted, actual)
MAE = mean_absolute_error(predicted, actual)
R2 = r2_score(actual, predicted)

print((MSE, MAE, R2))